In [ ]:
#GNN model

import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from sklearn.model_selection import train_test_split

# Define the GNN model
class GNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        #self.conv2 = GCNConv(hidden_channels1, hidden_channels2)
        self.conv3 = GCNConv(hidden_channels, out_channels)


    def forward(self, x, edge_index, edge_attr):
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        #x = self.conv2(x, edge_index, edge_attr)
        #x = F.relu(x)
        x = self.conv3(x, edge_index, edge_attr)
        return x 

# Assuming gnn_dataset is your dataset created in the previous code
# Split the dataset into training and testing sets (80:20 split)
train_dataset, test_dataset = train_test_split(gnn_dataset, test_size=0.2, random_state=42)

# Create data loaders
train_loader = torch_geometric.loader.DataLoader(train_dataset, batch_size=40, shuffle=False)
test_loader = torch_geometric.loader.DataLoader(test_dataset, batch_size=40, shuffle=False)

# Initialize the model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_channels=gnn_dataset[0].x.shape[1], hidden_channels=128, out_channels=gnn_dataset[0].x.shape[1]).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss()

# Training loop
def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(train_loader)

# Example training loop
model.train()
ctr = 0
for epoch in range(10): # Number of epochs
  for data in gnn_dataset:
      optimizer.zero_grad()
      out = model(data.x, data.edge_index, data.edge_attr)
      if ctr ==7896:
         print(f"prediction: {out}")
         print(f"label: {data.y}")
      loss = criterion(out, data.y)
      loss.backward()
      optimizer.step()
    

      ctr=ctr+1
  print(f'Epoch: {epoch}, Loss: {loss}')

# Testing loop
def test(loader):
    model.eval()
    correct = 0
    for data in test_loader:
        out = model(data.x, data.edge_index, data.edge_attr)
        pred = out        
        correct += criterion(pred,data.y)
    return correct / len(loader.dataset)


# Training and evaluation
for epoch in range(1, 10):  # Adjust number of epochs
    loss = train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch}, Loss: {loss:.4f}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')